In [ ]:
# Output saving config
OUTPUT_DIR = 'outputs'
from utils import ensure_dir
ensure_dir(OUTPUT_DIR)


# Experiment 5: Prompt Ablation Study

Evaluate impact of increasingly constrained prompts on validity & action quality.

In [ ]:
MODEL_KEY = 'qwen25'
PROMPTS = {
  'minimal': 'What should the robot do?',
  'structured': 'You are a driving assistant. Output ONLY JSON {"action": "FORWARD|LEFT|RIGHT|STOP|SLOW", "rationale": "<short reason>"}.',
  'rich': 'Fuse perception. Output ONLY JSON with action rationale referencing lane & obstacles: {"action":...,"rationale":...}'
}

In [ ]:
from PIL import Image
from utils import list_frame_pairs, generate_action, parse_action_json
pairs = list_frame_pairs()
len(pairs)

In [ ]:
results = {}
all_outputs=[]
for label,prompt in PROMPTS.items():
    parsed = []
    for cam_path,_ in pairs:
        img = Image.open(cam_path).convert('RGB')
        out = generate_action(MODEL_KEY, [img], prompt)
        pj = parse_action_json(out) or {}
        parsed.append(pj)
        all_outputs.append(out)
    validity = sum(1 for p in parsed if 'action' in p)/len(parsed)
    results[label] = validity
results

In [ ]:
# Persist prompt validity results
if 'results' in globals():
    import json, os
    from utils import save_text
    save_text(os.path.join(OUTPUT_DIR,'experiment5_prompt_ablation_summary.json'), json.dumps(results, indent=2))
    results
else:
    print('Run ablation cell first.')